In [1]:
import numpy as np
import reader

## Read in  spam and ham data

In [2]:
def load_data(trainingdir, stemming=False, lowercase=False, silently=False):
    print(f"Stemming is {stemming}")
    print(f"Lowercase is {lowercase}")
    train_set, train_labels = reader.load_dataset_main(trainingdir,stemming,lowercase,silently)
    return train_set, train_labels

In [3]:
training_dir = "data"

In [4]:
train_set, train_labels = load_data(training_dir)

Stemming is False
Lowercase is False


In [10]:
len(train_set)
len(train_labels)

5172